<a href="https://colab.research.google.com/github/arthursl12/POC1/blob/main/POC1_Turbofan_FD001_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import glob

In [2]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import SGDRegressor
from sklearn.svm import LinearSVR

from sklearn.pipeline import Pipeline
from sklearn.compose import TransformedTargetRegressor
from sklearn.exceptions import ConvergenceWarning

from sklearn.feature_selection import SelectFromModel
from sklearn.tree import DecisionTreeRegressor

from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

In [3]:
sns.set_palette('colorblind')

# Data Preparation

In [4]:
# Dataset Download 
os.system('git clone https://github.com/arthursl12/dataset_2')
os.system('mv /content/dataset_2/CMaps /content/CMaps')
os.system('mv /content/dataset_2/data_processing /content/data_processing')
os.system('rm -rf dataset_2')

0

In [5]:
from data_processing.processing import DatasetProcessing
from data_processing.training import HyperparameterSearch, reclipper_scorer
from data_processing.eval import Evaluation

In [6]:
proc = DatasetProcessing()

## Data Integration

The data are provided as a zip-compressed text file with 26 columns of numbers, separated by spaces. Each row is a snapshot of data taken during a single operational cycle, each column is a different variable. The columns correspond to:  

1) unit number   
2) time, in cycles  
3) operational setting 1  
4) operational setting 2  
5) operational setting 3    
6) sensor measurement 1    
7) sensor measurement 2  
...  
26) sensor measurement 20


There are 6 conditions (or combinations) which the 3 operational settings can take.  
Condition 1: Altitude = 0, Mach Number = 0, TRA = 100  
Condition 2: Altitude = 10, Mach Number = 0.25, TRA = 100  
Condition 3: Altitude = 20, Mach Number = 0.7 TRA = 100  
Condition 4: Altitude = 25, Mach Number = 0.62, TRA = 60  
Condition 5: Altitude = 35 Mach Number = 0.84, TRA = 100  
Condition 6: Altitude = 42, Mach Number = 0.84, TRA = 100  
  
There is slight variation in all these conditions so you may get numbers like 24.453 instead of 25 exactly.

FD001: Condition 1 only  
FD002: Mix of all the conditions  
FD003: Condition 1 only  
FD004: Mix of all conditions  


In [7]:
index_cols, settings_cols, sensors_cols, cols = proc.column_names()
train, test, y_test = proc.read_dataset(1)
train

,unit_number,time,op_1,op_2,op_3,s_0,s_1,s_2,s_3,s_4,...,s_11,s_12,s_13,s_14,s_15,s_16,s_17,s_18,s_19,s_20
0,1,1,-0.0007,-0.0004,100.0,518.67,641.82,1589.70,1400.60,14.62,...,521.66,2388.02,8138.62,8.4195,0.03,392,2388,100.0,39.06,23.4190
1,1,2,0.0019,-0.0003,100.0,518.67,642.15,1591.82,1403.14,14.62,...,522.28,2388.07,8131.49,8.4318,0.03,392,2388,100.0,39.00,23.4236
2,1,3,-0.0043,0.0003,100.0,518.67,642.35,1587.99,1404.20,14.62,...,522.42,2388.03,8133.23,8.4178,0.03,390,2388,100.0,38.95,23.3442
3,1,4,0.0007,0.0000,100.0,518.67,642.35,1582.79,1401.87,14.62,...,522.86,2388.08,8133.83,8.3682,0.03,392,2388,100.0,38.88,23.3739
4,1,5,-0.0019,-0.0002,100.0,518.67,642.37,1582.85,1406.22,14.62,...,522.19,2388.04,8133.80,8.4294,0.03,393,2388,100.0,38.90,23.4044
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20626,100,196,-0.0004,-0.0003,100.0,518.67,643.49,1597.98,1428.63,14.62,...,519.49,2388.26,8137.60,8.4956,0.03,397,2388,100.0,38.49,22.9735
20627,100,197,-0.0016,-0.0005,100.0,518.67,643.54,1604.50,1433.58,14.62,...,519.68,2388.22,8136.50,8.5139,0.03,395,2388,100.0,38.30,23.1594
20628,100,198,0.0004,0.0000,100.0,518.67,643.42,1602.46,1428.18,14.62,...,520.01,2388.24,8141.05,8.5646,0.03,398,2388,100.0,38.44,22.9333
20629,100,199,-0.0011,0.0003,100.0,518.67,643.23,1605.26,1426.53,14.62,...,519.67,2388.23,8139.29,8.5389,0.03,395,2388,100.0,38.29,23.0640


## Preprocessing

### Test Set Transformation 
Test set has samples for all cycles, but has annotations only for last one

In [8]:
test.shape, y_test.shape

((13096, 26), (100, 1))

In [9]:
test_last = proc.transform_test(test)
test_last.head()

,s_0,s_1,s_2,s_3,s_4,s_5,s_6,s_7,s_8,s_9,...,s_11,s_12,s_13,s_14,s_15,s_16,s_17,s_18,s_19,s_20
0,518.67,642.58,1581.22,1398.91,14.62,21.61,554.42,2388.08,9056.40,1.3,...,521.79,2388.06,8130.11,8.4024,0.03,393,2388,100.0,38.81,23.3552
1,518.67,642.55,1586.59,1410.83,14.62,21.61,553.52,2388.10,9044.77,1.3,...,521.74,2388.09,8126.90,8.4505,0.03,391,2388,100.0,38.81,23.2618
2,518.67,642.88,1589.75,1418.89,14.62,21.61,552.59,2388.16,9049.26,1.3,...,520.83,2388.14,8131.46,8.4119,0.03,395,2388,100.0,38.93,23.2740
3,518.67,642.78,1594.53,1406.88,14.62,21.61,552.64,2388.13,9051.30,1.3,...,521.88,2388.11,8133.64,8.4634,0.03,395,2388,100.0,38.58,23.2581
4,518.67,642.27,1589.94,1419.36,14.62,21.61,553.29,2388.10,9053.99,1.3,...,521.00,2388.15,8125.74,8.4362,0.03,394,2388,100.0,38.75,23.4117


In [10]:
X_test = test_last

### Remaining Useful Life (RUL)

In [11]:
train = proc.add_remaining_useful_life_linear(train)
train[index_cols+['RUL']].head()

,unit_number,time,RUL
0,1,1,191
1,1,2,190
2,1,3,189
3,1,4,188
4,1,5,187


## Attributes and target separation

In [12]:
X_train, y_train = proc.X_y_train_divide(train)

In [13]:
y_train.head()

,RUL
0,191
1,190
2,189
3,188
4,187


In [14]:
X_train.head()

,s_0,s_1,s_2,s_3,s_4,s_5,s_6,s_7,s_8,s_9,...,s_11,s_12,s_13,s_14,s_15,s_16,s_17,s_18,s_19,s_20
0,518.67,641.82,1589.70,1400.60,14.62,21.61,554.36,2388.06,9046.19,1.3,...,521.66,2388.02,8138.62,8.4195,0.03,392,2388,100.0,39.06,23.4190
1,518.67,642.15,1591.82,1403.14,14.62,21.61,553.75,2388.04,9044.07,1.3,...,522.28,2388.07,8131.49,8.4318,0.03,392,2388,100.0,39.00,23.4236
2,518.67,642.35,1587.99,1404.20,14.62,21.61,554.26,2388.08,9052.94,1.3,...,522.42,2388.03,8133.23,8.4178,0.03,390,2388,100.0,38.95,23.3442
3,518.67,642.35,1582.79,1401.87,14.62,21.61,554.45,2388.11,9049.48,1.3,...,522.86,2388.08,8133.83,8.3682,0.03,392,2388,100.0,38.88,23.3739
4,518.67,642.37,1582.85,1406.22,14.62,21.61,554.00,2388.06,9055.15,1.3,...,522.19,2388.04,8133.80,8.4294,0.03,393,2388,100.0,38.90,23.4044


## Training and Evaluation functions

In [15]:
eval = Evaluation()

In [16]:
search = HyperparameterSearch()

# LinearRegression

## Linear RUL

In [ ]:
model = LinearRegression()

In [ ]:
model.fit(X_train, y_train)
eval.show_result(y_train, model.predict(X_train))
eval.show_result_cv(y_train, X_train, model)

R2=0.580,RMSE=-44.663
(CV) R2=0.569,RMSE=-44.666


In [ ]:
eval.show_result(y_test, model.predict(X_test))

R2=0.409,RMSE=-31.950


## Non-linear RUL

In [ ]:
y_train.clip(upper=189).head()

,RUL
0,189
1,189
2,189
3,188
4,187


In [ ]:
model = Pipeline([
    ('trf_reg' ,TransformedTargetRegressor(
        check_inverse=False,
        regressor   = LinearRegression(),
        transformer = FunctionTransformer(np.clip, 
                                          kw_args={'a_min':0,'a_max':50})))
    ])

In [ ]:
GRID_SEARCH = False
if (GRID_SEARCH):
    param_distributions = {
        "trf_reg__transformer__kw_args": search.generate_clip_dicts(80,150,1)
    }
    model = search.run_HR_GS(model, X_train, y_train, param_distributions)
    print(model)

Pipeline(steps=[('trf_reg',
                 TransformedTargetRegressor(check_inverse=False,
                                            regressor=LinearRegression(),
                                            transformer=FunctionTransformer(func=<function clip at 0x7f9de2041cb0>,
                                                                            kw_args={'a_max': 50,
                                                                                     'a_min': 0})))])

Best Model:

```
{'trf_reg__transformer__kw_args': {'a_min': 0, 'a_max': 96}}
```



In [ ]:
model = Pipeline([
    ('trf_reg' ,TransformedTargetRegressor(
        check_inverse=False,
        regressor   = LinearRegression(),
        transformer = FunctionTransformer(np.clip, 
                                          kw_args={'a_min':0,'a_max':96})))
    ])
model

In [ ]:
model.fit(X_train, y_train)
reclipped_y =  model['trf_reg'].transformer.transform(y_train)
eval.show_result(reclipped_y, model.predict(X_train))
eval.show_result_cv(reclipped_y, X_train, model)

R2=0.757,RMSE=-15.128
(CV) R2=0.753,RMSE=-15.224


In [ ]:
reclipped_y =  model['trf_reg'].transformer.transform(y_test)
eval.show_result(reclipped_y, model.predict(X_test))

R2=0.623,RMSE=-25.526


## PolyFeatures + Linear RUL

In [ ]:
model = Pipeline([
    ('poly_ft'  ,   PolynomialFeatures()),
    ('lin_reg'  ,   LinearRegression())
])
model

Pipeline(steps=[('poly_ft', PolynomialFeatures()),
                ('lin_reg', LinearRegression())])

In [ ]:
GRID_SEARCH = False
if (GRID_SEARCH):
    param_distributions = {
        "poly_ft__degree": [1,2,3],
        "poly_ft__interaction_only": [False, True],
        "poly_ft__include_bias": [True, False]
    }
    model = search.run_HR_GS(model, X_train, y_train, 
                            param_distributions, scorer='r2')
    print(model)

Best Model:
```
{'poly_ft__interaction_only': False, 
'poly_ft__include_bias': False, 
'poly_ft__degree': 1}
```


In [ ]:
model = Pipeline([
    ('poly_ft'  ,   PolynomialFeatures(degree=1, include_bias=False)),
    ('lin_reg'  ,   LinearRegression())
])
model

Pipeline(steps=[('poly_ft', PolynomialFeatures(degree=1, include_bias=False)),
                ('lin_reg', LinearRegression())])

In [ ]:
model.fit(X_train, y_train)
eval.show_result(y_train, model.predict(X_train))
eval.show_result_cv(y_train, X_train, model)

R2=0.580,RMSE=-44.663
(CV) R2=0.569,RMSE=-44.666


In [ ]:
eval.show_result(y_test, model.predict(X_test))

R2=0.409,RMSE=-31.950


No clear benefit from polynomial features in linear RUL for simple linear regression. Actually, the degree stayed in 1, i.e., without those features.

## PolyFeatures + Non-Linear RUL

In [ ]:
model = Pipeline([
    ('poly_ft'  ,   PolynomialFeatures()),
    ('trf_reg' ,TransformedTargetRegressor(
        check_inverse=False,
        regressor   = LinearRegression(),
        transformer = FunctionTransformer(np.clip, 
                                          kw_args={'a_min':0,'a_max':96})))
])

Pipeline(steps=[('poly_ft', PolynomialFeatures()),
                ('trf_reg',
                 TransformedTargetRegressor(check_inverse=False,
                                            regressor=LinearRegression(),
                                            transformer=FunctionTransformer(func=<function clip at 0x7f9de2041cb0>,
                                                                            kw_args={'a_max': 96,
                                                                                     'a_min': 0})))])

In [ ]:
GRID_SEARCH = False
if (GRID_SEARCH):
    param_distributions = {
        "poly_ft__degree": [1,2,3],
        "poly_ft__interaction_only": [False, True],
        "poly_ft__include_bias": [True, False],
        "trf_reg__transformer__kw_args": search.generate_clip_dicts(70,150,1)
    }
    model = search.run_HR_GS(model, X_train, y_train, param_distributions)
    print(model)

Best Model:
```
{'trf_reg__transformer__kw_args': {'a_min': 0, 'a_max': 93}, 
'poly_ft__interaction_only': False, 
'poly_ft__include_bias': True, 
'poly_ft__degree': 1}
```


In [ ]:
model = Pipeline([
    ('poly_ft'  ,   PolynomialFeatures(degree=1)),
    ('trf_reg' ,TransformedTargetRegressor(
        check_inverse=False,
        regressor   = LinearRegression(),
        transformer = FunctionTransformer(np.clip, 
                                          kw_args={'a_min':0,'a_max':93})))
])
model

Pipeline(steps=[('poly_ft', PolynomialFeatures(degree=1)),
                ('trf_reg',
                 TransformedTargetRegressor(check_inverse=False,
                                            regressor=LinearRegression(),
                                            transformer=FunctionTransformer(func=<function clip at 0x7f9de2041cb0>,
                                                                            kw_args={'a_max': 93,
                                                                                     'a_min': 0})))])

In [ ]:
model.fit(X_train, y_train)
reclipped_y =  model['trf_reg'].transformer.transform(y_train)
eval.show_result(reclipped_y, model.predict(X_train))
eval.show_result_cv(reclipped_y, X_train, model)

R2=0.758,RMSE=-14.536
(CV) R2=0.753,RMSE=-14.630


In [ ]:
reclipped_y =  model['trf_reg'].transformer.transform(y_test)
eval.show_result(reclipped_y, model.predict(X_test))

R2=0.596,RMSE=-26.415


No clear benefit from polynomial features in non-linear RUL for simple linear regression. Actually, the degree stayed in 1, i.e., without those features.

# SGD Regressor

The class SGDRegressor implements a plain stochastic gradient descent learning routine which supports different loss functions and penalties to fit linear regression models. **SGDRegressor is well suited for regression problems with a large number of training samples (> 10.000)**, for other problems we recommend Ridge, Lasso, or ElasticNet.

## Linear RUL

In [ ]:
model = Pipeline([
    ('scaler'   ,   StandardScaler()),
    ('lin_reg'  ,   SGDRegressor())
])

In [17]:
# We need specific shape 1D arrays for this model
y_train = np.array(y_train).ravel()

In [ ]:
GRID_SEARCH = False
if (GRID_SEARCH):
    param_distributions = {
        "lin_reg__loss": ['squared_error', 'huber', 'epsilon_insensitive'],
        "lin_reg__penalty": ['l2', 'l1', 'elasticnet'],
        "lin_reg__epsilon": [1.1,1.35,1.5,2,2.5,3,3.5,4,4.5,5.5,6],
        "lin_reg__alpha":[1e-2,1e-3,1e-4,1e-5,1e-6,1e-7],
        "lin_reg__max_iter": [1000,1500,2000,2500,3000],
        "lin_reg__tol": [1e-3,1e-4,1e-5,1e-6,1e-7],
        "lin_reg__learning_rate":['invscaling','constant','optimal','adaptive'],
        "lin_reg__eta0": [1e-1,1e-2,1e-3,1e-4,1e-5,1e-6],
        "lin_reg__power_t": [1.25,1,0.75,0.5,0.25,0.1],
        "lin_reg__early_stopping": [True,False],
        "lin_reg__validation_fraction": [0.1,0.2,0.3,0.4,0.5,0.6],
        "lin_reg__n_iter_no_change": [1,2,5,8,10,15,20],
        "lin_reg__average": [False, 1,2,5,10,20]
    }
    model = search.run_HR_GS(model, X_train, y_train, param_distributions, 
                            scorer='r2', ignore_warnings=True)
    print(model)

Best Model:
```
SGDRegressor(average=10, epsilon=4, eta0=1e-05,
            learning_rate='adaptive', max_iter=1500,
            n_iter_no_change=8, penalty='elasticnet',
            power_t=0.1, tol=1e-06,
            validation_fraction=0.5)
```

In [ ]:
model = Pipeline([
    ('scaler'   ,   StandardScaler()),
    ('lin_reg'  ,   SGDRegressor(average=10, epsilon=4, eta0=1e-05,
                                 learning_rate='adaptive', max_iter=1500,
                                 n_iter_no_change=8, penalty='elasticnet',
                                 power_t=0.1, tol=1e-06,
                                 validation_fraction=0.5))
])
model

Pipeline(steps=[('scaler', StandardScaler()),
                ('lin_reg',
                 SGDRegressor(average=10, epsilon=4, eta0=1e-05,
                              learning_rate='adaptive', max_iter=1500,
                              n_iter_no_change=8, penalty='elasticnet',
                              power_t=0.1, tol=1e-06,
                              validation_fraction=0.5))])

In [ ]:
model.fit(X_train, y_train)
eval.show_result(y_train, model.predict(X_train))
eval.show_result_cv(y_train, X_train, model)

R2=0.578,RMSE=-44.732
(CV) R2=0.573,RMSE=-44.555


In [ ]:
X_test = test_last
eval.show_result(y_test, model.predict(X_test))

R2=0.472,RMSE=-30.203


Poor results without non-lineariy of RUL. Comparable to regular LinearRegressor.

## Non-linear RUL

In [ ]:
model = Pipeline([
    ('scaler'   ,   StandardScaler()),
    ('trf_reg' ,TransformedTargetRegressor(
        check_inverse=False,
        regressor   = SGDRegressor(),
        transformer = FunctionTransformer(np.clip, 
                                          kw_args={'a_min':0,'a_max':50})))
    ])

In [ ]:
GRID_SEARCH = False
if (GRID_SEARCH):
    param_distributions = {
        "trf_reg__transformer__kw_args": search.generate_clip_dicts(80,150,1),
        "trf_reg__regressor__loss": ['squared_error', 'huber', 'epsilon_insensitive'],
        "trf_reg__regressor__penalty": ['l2', 'l1', 'elasticnet'],
        "trf_reg__regressor__epsilon": [1.1,1.35,1.5,2,2.5,3,3.5,4,4.5,5.5,6],
        "trf_reg__regressor__alpha":[1e-2,1e-3,1e-4,1e-5,1e-6,1e-7],
        "trf_reg__regressor__max_iter": [1000,1500,2000,2500,3000],
        "trf_reg__regressor__tol": [1e-3,1e-4,1e-5,1e-6,1e-7],
        "trf_reg__regressor__learning_rate":['invscaling','constant','optimal','adaptive'],
        "trf_reg__regressor__eta0": [1e-1,1e-2,1e-3,1e-4,1e-5,1e-6],
        "trf_reg__regressor__power_t": [1.25,1,0.75,0.5,0.25,0.1],
        "trf_reg__regressor__early_stopping": [True,False],
        "trf_reg__regressor__validation_fraction": [0.1,0.2,0.3,0.4,0.5,0.6],
        "trf_reg__regressor__n_iter_no_change": [1,2,5,8,10,15,20],
        "trf_reg__regressor__average": [False, 1,2,5,10,20]
    }
    model = search.run_HR_GS(model, X_train, y_train, 
                            param_distributions, ignore_warnings=True)
    print(model)

Best Model:
```
kw_args={'a_max': 104,'a_min': 0}))

SGDRegressor(alpha=0.001,
            early_stopping=True,
            epsilon=5.5,
            eta0=0.001,
            learning_rate='adaptive',
            max_iter=1500,
            power_t=1,
            tol=1e-05,
            validation_fraction=0.6),
```

In [ ]:
model = Pipeline([
    ('scaler'   ,   StandardScaler()),
    ('trf_reg' ,TransformedTargetRegressor(
        check_inverse=False,
        regressor=SGDRegressor(alpha=0.001, early_stopping=True,epsilon=5.5,
                               eta0=0.001, learning_rate='adaptive',
                               max_iter=1500, power_t=1, tol=1e-05,
                               validation_fraction=0.6),
        transformer = FunctionTransformer(np.clip, 
                                          kw_args={'a_min':0,'a_max':104})))
    ])
model

Pipeline(steps=[('scaler', StandardScaler()),
                ('trf_reg',
                 TransformedTargetRegressor(check_inverse=False,
                                            regressor=SGDRegressor(alpha=0.001,
                                                                   early_stopping=True,
                                                                   epsilon=5.5,
                                                                   eta0=0.001,
                                                                   learning_rate='adaptive',
                                                                   max_iter=1500,
                                                                   power_t=1,
                                                                   tol=1e-05,
                                                                   validation_fraction=0.6),
                                            transformer=FunctionTransformer(func=<function clip at 0x7f9de2041

In [ ]:
model.fit(X_train, y_train)
reclipped_y =  model['trf_reg'].transformer.transform(y_train)
eval.show_result(reclipped_y, model.predict(X_train))
eval.show_result_cv(reclipped_y, X_train, model)

R2=0.754,RMSE=-16.783
(CV) R2=0.750,RMSE=-16.869


In [ ]:
reclipped_y =  model['trf_reg'].transformer.transform(y_test)
eval.show_result(reclipped_y, model.predict(X_test))

R2=0.676,RMSE=-23.645


Achieved training results from non-linear RUL Linear Regressor, but with a little better test result. Also, with a RUL clipping of 104, almost 10 cycles more. 

## PolyFeatures + Linear RUL

In [18]:
model = Pipeline([
    ('poly_ft'  ,   PolynomialFeatures()),
    ('scaler'   ,   StandardScaler()),
    ('lin_reg'  ,   SGDRegressor())
])

In [ ]:
# ~5min
GRID_SEARCH = False
if (GRID_SEARCH):
    param_distributions = {
        "poly_ft__degree": [1,2,3],
        "poly_ft__interaction_only": [False, True],
        "poly_ft__include_bias": [True, False],
        "lin_reg__loss": ['squared_error', 'huber', 'epsilon_insensitive'],
        "lin_reg__penalty": ['l2', 'l1', 'elasticnet'],
        "lin_reg__epsilon": [1.1,1.35,1.5,2,2.5,3,3.5,4,4.5,5.5,6],
        "lin_reg__alpha":[1e-2,1e-3,1e-4,1e-5,1e-6,1e-7],
        "lin_reg__max_iter": [1000,1500,2000,2500,3000],
        "lin_reg__tol": [1e-3,1e-4,1e-5,1e-6,1e-7],
        "lin_reg__learning_rate":['invscaling','constant','optimal','adaptive'],
        "lin_reg__eta0": [1e-1,1e-2,1e-3,1e-4,1e-5,1e-6],
        "lin_reg__power_t": [1.25,1,0.75,0.5,0.25,0.1],
        "lin_reg__early_stopping": [True,False],
        "lin_reg__validation_fraction": [0.1,0.2,0.3,0.4,0.5,0.6],
        "lin_reg__n_iter_no_change": [1,2,5,8,10,15,20],
        "lin_reg__average": [False, 1,2,5,10,20]
    }
    model = search.run_HR_GS(model, X_train, y_train, param_distributions,
                            ignore_warnings=True, scorer='r2')
    print(model)

Best Model:
```
Pipeline(steps=[('poly_ft', PolynomialFeatures(degree=3, include_bias=False)),
                ('scaler', StandardScaler()),
                ('lin_reg',
                 SGDRegressor(alpha=1e-06, average=1, epsilon=6, eta0=0.0001,
                              learning_rate='adaptive', max_iter=2000,
                              n_iter_no_change=10, penalty='elasticnet',
                              tol=0.0001))])
```


In [19]:
model = Pipeline([
    ('poly_ft', PolynomialFeatures(degree=3, include_bias=False)),
    ('scaler', StandardScaler()),
    ('lin_reg',  SGDRegressor(alpha=1e-06, average=1, epsilon=6, eta0=0.0001,
                              learning_rate='adaptive', max_iter=2000,
                              n_iter_no_change=10, penalty='elasticnet',
                              tol=0.0001))
])
model

Pipeline(steps=[('poly_ft', PolynomialFeatures(degree=3, include_bias=False)),
                ('scaler', StandardScaler()),
                ('lin_reg',
                 SGDRegressor(alpha=1e-06, average=1, epsilon=6, eta0=0.0001,
                              learning_rate='adaptive', max_iter=2000,
                              n_iter_no_change=10, penalty='elasticnet',
                              tol=0.0001))])

In [20]:
model.fit(X_train, y_train)
eval.show_result(y_train, model.predict(X_train))
eval.show_result_cv(y_train, X_train, model)

R2=0.593,RMSE=-43.941
(CV) R2=0.578,RMSE=-44.226


In [21]:
eval.show_result(y_test, model.predict(X_test))

R2=0.432,RMSE=-31.319


No clear benefit from polynomial features in linear RUL as well.

## PolyFeatures + Non-Linear RUL

In [ ]:
model = Pipeline([
    ('poly_ft'  ,   PolynomialFeatures()),
    ('scaler'   ,   StandardScaler()),
    ('trf_reg' ,TransformedTargetRegressor(
        check_inverse=False,
        regressor   = SGDRegressor(),
        transformer = FunctionTransformer(np.clip, 
                                          kw_args={'a_min':0,'a_max':96})))
])

In [ ]:
# ~5min
GRID_SEARCH = False
if (GRID_SEARCH):
    param_distributions = {
        "poly_ft__degree": [1,2,3],
        "poly_ft__interaction_only": [False, True],
        "poly_ft__include_bias": [True, False],
        "trf_reg__transformer__kw_args": search.generate_clip_dicts(70,150,1),
        "trf_reg__regressor__loss": ['squared_error', 'huber', 'epsilon_insensitive'],
        "trf_reg__regressor__penalty": ['l2', 'l1', 'elasticnet'],
        "trf_reg__regressor__epsilon": [1.1,1.35,1.5,2,2.5,3,3.5,4,4.5,5.5,6],
        "trf_reg__regressor__alpha":[1e-2,1e-3,1e-4,1e-5,1e-6,1e-7],
        "trf_reg__regressor__max_iter": [1000,1500,2000,2500,3000,4000,4500],
        "trf_reg__regressor__tol": [1e-3,1e-4,1e-5,1e-6,1e-7],
        "trf_reg__regressor__learning_rate":['invscaling','constant','optimal','adaptive'],
        "trf_reg__regressor__eta0": [1e-1,1e-2,1e-3,1e-4,1e-5,1e-6],
        "trf_reg__regressor__power_t": [1.25,1,0.75,0.5,0.25,0.1],
        "trf_reg__regressor__early_stopping": [True,False],
        "trf_reg__regressor__validation_fraction": [0.1,0.2,0.3,0.4,0.5,0.6],
        "trf_reg__regressor__n_iter_no_change": [1,2,5,8,10,15,20],
        "trf_reg__regressor__average": [False, 1,2,5,10,20]
    }
    model = search.run_HR_GS(model, X_train, y_train, 
                             param_distributions, ignore_warnings=True)
    print(model)

n_iterations: 4
n_required_iterations: 4
n_possible_iterations: 4
min_resources_: 500
max_resources_: 20631
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 41
n_resources: 500
Fitting 5 folds for each of 41 candidates, totalling 205 fits
----------
iter: 1
n_candidates: 14
n_resources: 1500
Fitting 5 folds for each of 14 candidates, totalling 70 fits
----------
iter: 2
n_candidates: 5
n_resources: 4500
Fitting 5 folds for each of 5 candidates, totalling 25 fits
----------
iter: 3
n_candidates: 2
n_resources: 13500
Fitting 5 folds for each of 2 candidates, totalling 10 fits
Best params:  {'trf_reg__transformer__kw_args': {'a_min': 0, 'a_max': 115}, 'trf_reg__regressor__validation_fraction': 0.5, 'trf_reg__regressor__tol': 1e-06, 'trf_reg__regressor__power_t': 0.25, 'trf_reg__regressor__penalty': 'elasticnet', 'trf_reg__regressor__n_iter_no_change': 15, 'trf_reg__regressor__max_iter': 2500, 'trf_reg__regressor__loss': 'squared_error', 'trf_reg__regressor__learnin

Pipeline(steps=[('poly_ft',
                 PolynomialFeatures(include_bias=False, interaction_only=True)),
                ('scaler', StandardScaler()),
                ('trf_reg',
                 TransformedTargetRegressor(check_inverse=False,
                                            regressor=SGDRegressor(alpha=1e-05,
                                                                   average=20,
                                                                   early_stopping=True,
                                                                   epsilon=1.5,
                                                                   max_iter=2500,
                                                                   n_iter_no_change=15,
                                                                   penalty='elasticnet',
                                                                   tol=1e-06,
                                                                   validation_fraction=0

Best Model:
```
{'trf_reg__transformer__kw_args': {'a_min': 0, 'a_max': 93}, 
'poly_ft__interaction_only': False, 
'poly_ft__include_bias': True, 
'poly_ft__degree': 1}
```


In [ ]:
model = Pipeline([
    ('poly_ft',PolynomialFeatures(include_bias=False, interaction_only=True)),
    ('scaler', StandardScaler()),
    ('trf_reg',
        TransformedTargetRegressor(
            check_inverse=False,
            regressor=SGDRegressor(alpha=1e-05,
                                   average=20,
                                   early_stopping=True,
                                   epsilon=1.5,
                                   max_iter=2500,
                                   n_iter_no_change=15,
                                   penalty='elasticnet',
                                   tol=1e-06,
                                   validation_fraction=0.5),
            transformer=FunctionTransformer(np.clip,
                                            kw_args={'a_max': 115,
                                                     'a_min': 0})))
])
model

Pipeline(steps=[('poly_ft',
                 PolynomialFeatures(include_bias=False, interaction_only=True)),
                ('scaler', StandardScaler()),
                ('trf_reg',
                 TransformedTargetRegressor(check_inverse=False,
                                            regressor=SGDRegressor(alpha=1e-05,
                                                                   average=20,
                                                                   early_stopping=True,
                                                                   epsilon=1.5,
                                                                   max_iter=2500,
                                                                   n_iter_no_change=15,
                                                                   penalty='elasticnet',
                                                                   tol=1e-06,
                                                                   validation_fraction=0

In [ ]:
model.fit(X_train, y_train)
reclipped_y =  model['trf_reg'].transformer.transform(y_train)
eval.show_result(reclipped_y, model.predict(X_train))
eval.show_result_cv(reclipped_y, X_train, model)

R2=0.746,RMSE=-19.165
(CV) R2=0.741,RMSE=-19.283


In [ ]:
reclipped_y =  model['trf_reg'].transformer.transform(y_test)
eval.show_result(reclipped_y, model.predict(X_test))

R2=0.715,RMSE=-22.171


With 2-degree terms, we increased the test performance. The training is similar to the previous cases, a little smaller, probably due to the random search.

# SVR

We'll use only the linear kernel, because the algorithm is quadratic on the number of samples. Otherwise, the training times get worser.

## Linear RUL

In [ ]:
model = Pipeline([
    ('scaler'   ,   MinMaxScaler()),
    ('svm_reg'  ,   LinearSVR(random_state=42))
])

In [ ]:
# We need specific shape 1D arrays for this model
y_train = np.array(y_train).ravel()

In [ ]:
GRID_SEARCH = True
if (GRID_SEARCH):
    param_distributions = {
        "scaler": [StandardScaler(), MinMaxScaler()],
        "svm_reg__loss": ["epsilon_insensitive","squared_epsilon_insensitive"],
        "svm_reg__fit_intercept": [True, False],
        "svm_reg__max_iter": [1000,3000,5000],
        "svm_reg__epsilon": [1,1e-1,1e-2,1e-3],
        "svm_reg__tol": [1e-3,1e-4,1e-5,1e-6,1e-7,1e-8],
        "svm_reg__C":[1,2,5,10]
    }
    model = search.run_HR_GS(model, X_train, y_train, param_distributions, 
                            scorer='r2', ignore_warnings=True)
    print(model)

n_iterations: 4
n_required_iterations: 4
n_possible_iterations: 4
min_resources_: 500
max_resources_: 20631
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 41
n_resources: 500
Fitting 5 folds for each of 41 candidates, totalling 205 fits
----------
iter: 1
n_candidates: 14
n_resources: 1500
Fitting 5 folds for each of 14 candidates, totalling 70 fits
----------
iter: 2
n_candidates: 5
n_resources: 4500
Fitting 5 folds for each of 5 candidates, totalling 25 fits
----------
iter: 3
n_candidates: 2
n_resources: 13500
Fitting 5 folds for each of 2 candidates, totalling 10 fits
Best params:  {'svm_reg__tol': 1e-07, 'svm_reg__max_iter': 3000, 'svm_reg__loss': 'squared_epsilon_insensitive', 'svm_reg__fit_intercept': True, 'svm_reg__epsilon': 0.001, 'svm_reg__C': 2, 'scaler': MinMaxScaler()}
Pipeline(steps=[('scaler', MinMaxScaler()),
                ('svm_reg',
                 LinearSVR(C=2, epsilon=0.001,
                           loss='squared_epsilon_insensitive'

Best Model:
```
('scaler', MinMaxScaler()),
('svm_reg',
    LinearSVR(C=2, epsilon=0.001,
            loss='squared_epsilon_insensitive', max_iter=3000,
            random_state=42, tol=1e-07))
```

In [ ]:
model = Pipeline([
    ('scaler'   ,   MinMaxScaler()),
    ('svm_reg'  ,   LinearSVR(C=2, epsilon=0.001,
            loss='squared_epsilon_insensitive', max_iter=3000,
            random_state=42, tol=1e-07))
])
model

Pipeline(steps=[('scaler', MinMaxScaler()),
                ('svm_reg',
                 LinearSVR(C=2, epsilon=0.001,
                           loss='squared_epsilon_insensitive', max_iter=3000,
                           random_state=42, tol=1e-07))])

In [ ]:
model.fit(X_train, y_train)
eval.show_result(y_train, model.predict(X_train))
eval.show_result_cv(y_train, X_train, model)

R2=0.580,RMSE=-44.663
(CV) R2=0.569,RMSE=-44.665


In [ ]:
eval.show_result(y_test, model.predict(X_test))

R2=0.409,RMSE=-31.951


Poor results without non-lineariy of RUL. Comparable to regular LinearRegressor.

## Non-linear RUL

In [ ]:
model = Pipeline([
    ('scaler'   ,   MinMaxScaler()),
    ('trf_reg' ,TransformedTargetRegressor(
        check_inverse=False,
        regressor   = LinearSVR(random_state=42),
        transformer = FunctionTransformer(np.clip, 
                                          kw_args={'a_min':0,'a_max':50})))
    ])

In [ ]:
# ~4min
GRID_SEARCH = True
if (GRID_SEARCH):
    param_distributions = {
        "scaler": [StandardScaler(), MinMaxScaler()],
        "trf_reg__transformer__kw_args": search.generate_clip_dicts(80,150,1),
        "trf_reg__regressor__loss": ["epsilon_insensitive","squared_epsilon_insensitive"],
        "trf_reg__regressor__fit_intercept": [True, False],
        "trf_reg__regressor__max_iter": [1000,3000,5000],
        "trf_reg__regressor__epsilon": [1,1e-1,1e-2,1e-3],
        "trf_reg__regressor__tol": [1e-3,1e-4,1e-5,1e-6,1e-7,1e-8],
        "trf_reg__regressor__C":[1,2,5,10]
    }
    model = search.run_HR_GS(model, X_train, y_train, 
                            param_distributions, ignore_warnings=True)
    print(model)

n_iterations: 4
n_required_iterations: 4
n_possible_iterations: 4
min_resources_: 500
max_resources_: 20631
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 41
n_resources: 500
Fitting 5 folds for each of 41 candidates, totalling 205 fits
----------
iter: 1
n_candidates: 14
n_resources: 1500
Fitting 5 folds for each of 14 candidates, totalling 70 fits
----------
iter: 2
n_candidates: 5
n_resources: 4500
Fitting 5 folds for each of 5 candidates, totalling 25 fits
----------
iter: 3
n_candidates: 2
n_resources: 13500
Fitting 5 folds for each of 2 candidates, totalling 10 fits
Best params:  {'trf_reg__transformer__kw_args': {'a_min': 0, 'a_max': 95}, 'trf_reg__regressor__tol': 0.001, 'trf_reg__regressor__max_iter': 1000, 'trf_reg__regressor__loss': 'squared_epsilon_insensitive', 'trf_reg__regressor__fit_intercept': True, 'trf_reg__regressor__epsilon': 0.001, 'trf_reg__regressor__C': 10, 'scaler': MinMaxScaler()}
Pipeline(steps=[('scaler', MinMaxScaler()),
         

Best Model:
```
kw_args={'a_max': 95,'a_min': 0}))
('scaler', MinMaxScaler())
LinearSVR(C=10,
        epsilon=0.001,
        loss='squared_epsilon_insensitive',
        random_state=42,
        tol=0.001),
```

In [ ]:
model = Pipeline([
    ('scaler'   ,   MinMaxScaler()),
    ('trf_reg' ,TransformedTargetRegressor(
        check_inverse=False,
        regressor=LinearSVR(C=10,
                    epsilon=0.001,
                    loss='squared_epsilon_insensitive',
                    random_state=42,
                    tol=0.001),
        transformer = FunctionTransformer(np.clip, 
                                          kw_args={'a_min':0,'a_max':95})))
    ])
model

Pipeline(steps=[('scaler', MinMaxScaler()),
                ('trf_reg',
                 TransformedTargetRegressor(check_inverse=False,
                                            regressor=LinearSVR(C=10,
                                                                epsilon=0.001,
                                                                loss='squared_epsilon_insensitive',
                                                                random_state=42,
                                                                tol=0.001),
                                            transformer=FunctionTransformer(func=<function clip at 0x7f48ef421cb0>,
                                                                            kw_args={'a_max': 95,
                                                                                     'a_min': 0})))])

In [ ]:
model.fit(X_train, y_train)
reclipped_y =  model['trf_reg'].transformer.transform(y_train)
eval.show_result(reclipped_y, model.predict(X_train))
eval.show_result_cv(reclipped_y, X_train, model)

R2=0.757,RMSE=-14.929
(CV) R2=0.753,RMSE=-15.025


In [ ]:
reclipped_y =  model['trf_reg'].transformer.transform(y_test)
eval.show_result(reclipped_y, model.predict(X_test))

R2=0.614,RMSE=-25.807


Good linear results, but didn't generalize that well

## PolyFeatures + Linear RUL

In [ ]:
model = Pipeline([
    ('poly_ft'  ,   PolynomialFeatures()),
    ('selection', SelectFromModel(DecisionTreeRegressor())),
    ('scaler'   ,   MinMaxScaler()),
    ('svm_reg'  ,   LinearSVR(random_state=42))
])

In [ ]:
# ~20min
GRID_SEARCH = False
if (GRID_SEARCH):
    param_distributions = {
        "scaler": [StandardScaler(), MinMaxScaler()],
        "poly_ft__degree": [1,2,3],
        "poly_ft__interaction_only": [False, True],
        "poly_ft__include_bias": [True, False],
        "svm_reg__loss": ["epsilon_insensitive","squared_epsilon_insensitive"],
        "svm_reg__fit_intercept": [True, False],
        "svm_reg__max_iter": [1000,3000,5000],
        "svm_reg__epsilon": [1,1e-1,1e-2,1e-3],
        "svm_reg__tol": [1e-3,1e-4,1e-5,1e-6,1e-7,1e-8],
        "svm_reg__C":[1,2,5,10]
    }
    model = search.run_HR_GS(model, X_train, y_train, param_distributions,
                            ignore_warnings=True, scorer='r2')
    print(model)

n_iterations: 4
n_required_iterations: 4
n_possible_iterations: 4
min_resources_: 500
max_resources_: 20631
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 41
n_resources: 500
Fitting 5 folds for each of 41 candidates, totalling 205 fits
----------
iter: 1
n_candidates: 14
n_resources: 1500
Fitting 5 folds for each of 14 candidates, totalling 70 fits
----------
iter: 2
n_candidates: 5
n_resources: 4500
Fitting 5 folds for each of 5 candidates, totalling 25 fits
----------
iter: 3
n_candidates: 2
n_resources: 13500
Fitting 5 folds for each of 2 candidates, totalling 10 fits
Best params:  {'svm_reg__tol': 1e-05, 'svm_reg__max_iter': 3000, 'svm_reg__loss': 'squared_epsilon_insensitive', 'svm_reg__fit_intercept': True, 'svm_reg__epsilon': 1, 'svm_reg__C': 2, 'scaler': StandardScaler(), 'poly_ft__interaction_only': True, 'poly_ft__include_bias': False, 'poly_ft__degree': 3}
Pipeline(steps=[('poly_ft',
                 PolynomialFeatures(degree=3, include_bias=False,

Best Model:
```
PolynomialFeatures(degree=3, include_bias=False,
                                    interaction_only=True))
('selection',
                 SelectFromModel(estimator=DecisionTreeRegressor())),
('scaler', StandardScaler()),
LinearSVR(C=2, epsilon=1, loss='squared_epsilon_insensitive',
                           max_iter=3000, random_state=42, tol=1e-05))
```


In [ ]:
model = Pipeline([
    ('poly_ft', PolynomialFeatures(degree=3, include_bias=False,
                                    interaction_only=True)),
    ('selection', SelectFromModel(DecisionTreeRegressor())),
    ('scaler', StandardScaler()),
    ('svm_reg', LinearSVR(C=2, epsilon=1, loss='squared_epsilon_insensitive',
                           max_iter=3000, random_state=42, tol=1e-05))
])
model

Pipeline(steps=[('poly_ft',
                 PolynomialFeatures(degree=3, include_bias=False,
                                    interaction_only=True)),
                ('selection',
                 SelectFromModel(estimator=DecisionTreeRegressor())),
                ('scaler', StandardScaler()),
                ('svm_reg',
                 LinearSVR(C=2, epsilon=1, loss='squared_epsilon_insensitive',
                           max_iter=3000, random_state=42, tol=1e-05))])

In [ ]:
model.fit(X_train, y_train)
eval.show_result(y_train, model.predict(X_train))
eval.show_result_cv(y_train, X_train, model)

/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,


R2=0.622,RMSE=-42.333


/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number

(CV) R2=0.615,RMSE=-42.211


/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,


In [ ]:
eval.show_result(y_test, model.predict(X_test))

R2=0.413,RMSE=-31.831


No clear benefit from polynomial features in linear RUL as well. It didn't converge.

## PolyFeatures + Non-Linear RUL

In [ ]:
model = Pipeline([
    ('poly_ft'  ,   PolynomialFeatures()),
    ('selection', SelectFromModel(DecisionTreeRegressor())),
    ('scaler'   ,   StandardScaler()),
    ('trf_reg' ,TransformedTargetRegressor(
        check_inverse=False,
        regressor   = LinearSVR(random_state=42),
        transformer = FunctionTransformer(np.clip, 
                                          kw_args={'a_min':0,'a_max':96})))
])

In [ ]:
# ~15min
GRID_SEARCH = False
if (GRID_SEARCH):
    param_distributions = {
        "scaler": [StandardScaler(), MinMaxScaler()],
        "trf_reg__transformer__kw_args": search.generate_clip_dicts(80,150,1),
        "trf_reg__regressor__loss": ["epsilon_insensitive","squared_epsilon_insensitive"],
        "trf_reg__regressor__fit_intercept": [True, False],
        "trf_reg__regressor__max_iter": [1000,3000,5000],
        "trf_reg__regressor__epsilon": [1,1e-1,1e-2,1e-3],
        "trf_reg__regressor__tol": [1e-3,1e-4,1e-5,1e-6,1e-7,1e-8],
        "trf_reg__regressor__C":[1,2,5,10],
        "poly_ft__degree": [1,2,3],
        "poly_ft__interaction_only": [False, True],
        "poly_ft__include_bias": [True, False],
    }
    model = search.run_HR_GS(model, X_train, y_train, 
                             param_distributions, ignore_warnings=True)
    print(model)

n_iterations: 4
n_required_iterations: 4
n_possible_iterations: 4
min_resources_: 500
max_resources_: 20631
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 41
n_resources: 500
Fitting 5 folds for each of 41 candidates, totalling 205 fits
----------
iter: 1
n_candidates: 14
n_resources: 1500
Fitting 5 folds for each of 14 candidates, totalling 70 fits
----------
iter: 2
n_candidates: 5
n_resources: 4500
Fitting 5 folds for each of 5 candidates, totalling 25 fits
----------
iter: 3
n_candidates: 2
n_resources: 13500
Fitting 5 folds for each of 2 candidates, totalling 10 fits
Best params:  {'trf_reg__transformer__kw_args': {'a_min': 0, 'a_max': 93}, 'trf_reg__regressor__tol': 0.001, 'trf_reg__regressor__max_iter': 3000, 'trf_reg__regressor__loss': 'epsilon_insensitive', 'trf_reg__regressor__fit_intercept': True, 'trf_reg__regressor__epsilon': 1, 'trf_reg__regressor__C': 10, 'scaler': StandardScaler(), 'poly_ft__interaction_only': False, 'poly_ft__include_bias': Tr

Best Model:
```
{'trf_reg__transformer__kw_args': {'a_min': 0, 'a_max': 93}, 
PolynomialFeatures(degree=3)
('scaler', StandardScaler()),
LinearSVR(C=10, epsilon=1,
        max_iter=3000,
        random_state=42,
        tol=0.001),
```


In [ ]:
model = Pipeline([
    ('poly_ft',PolynomialFeatures(degree=3)),
    ('selection',
                 SelectFromModel(estimator=DecisionTreeRegressor())),
    ('scaler', StandardScaler()),
    ('trf_reg',
        TransformedTargetRegressor(
            check_inverse=False,
            regressor=LinearSVR(C=10, epsilon=1,
                                max_iter=3000,
                                random_state=42,
                                tol=0.001),
            transformer=FunctionTransformer(np.clip,
                                            kw_args={'a_max': 93,
                                                     'a_min': 0})))
])
model

Pipeline(steps=[('poly_ft', PolynomialFeatures(degree=3)),
                ('selection',
                 SelectFromModel(estimator=DecisionTreeRegressor())),
                ('scaler', StandardScaler()),
                ('trf_reg',
                 TransformedTargetRegressor(check_inverse=False,
                                            regressor=LinearSVR(C=10, epsilon=1,
                                                                max_iter=3000,
                                                                random_state=42,
                                                                tol=0.001),
                                            transformer=FunctionTransformer(func=<function clip at 0x7f48ef421cb0>,
                                                                            kw_args={'a_max': 93,
                                                                                     'a_min': 0})))])

In [ ]:
# ~10min
model.fit(X_train, y_train)
reclipped_y =  model['trf_reg'].transformer.transform(y_train)
eval.show_result(reclipped_y, model.predict(X_train))
eval.show_result_cv(reclipped_y, X_train, model)

/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,


R2=0.806,RMSE=-13.010
(CV) R2=0.799,RMSE=-13.185


In [ ]:
reclipped_y =  model['trf_reg'].transformer.transform(y_test)
eval.show_result(reclipped_y, model.predict(X_test))

R2=0.789,RMSE=-14.641


There was a case of failing to converge. But we achieved the best results in testing using linear kernel models.